## Bibliotecas a Utilizar

In [1]:
import numpy as np
import pandas as pd

## Enlaces

In [2]:
l_enlaces = ['./data/2018_06_Indicadores_Gestion.xls', './data/2018_03_Indicadores_Gestion.xls',
           './data/2017_12_Indicadores_Gestion.xls', './data/2017_09_Indicadores_Gestion.xls', './data/2017_06_Indicadores_Gestion.xls', './data/2017_03_Indicadores_Gestion.xls',
           './data/2016_12_Indicadores_Gestion.xls', './data/2016_09_Indicadores_Gestion.xls', './data/2016_06_Indicadores_Gestion.xls', './data/2016_03_Indicadores_Gestion.xls',
           './data/2015_12_Indicadores_Gestion.xls', './data/2015_09_Indicadores_Gestion.xls', './data/2015_06_Indicadores_Gestion.xls', './data/2015_03_Indicadores_Gestion.xls',
           './data/2014_12_Indicadores_Gestion.xls', './data/2014_09_Indicadores_Gestion.xls', './data/2014_06_Indicadores_Gestion.xls', './data/2014_03_Indicadores_Gestion.xls',
           './data/2013_12_Indicadores_Gestion.xls', './data/2013_09_Indicadores_Gestion.xls', './data/2013_06_Indicadores_Gestion.xls', './data/2013_03_Indicadores_Gestion.xls',
           './data/2012_12_Indicadores_Gestion.xls', './data/2012_09_Indicadores_Gestion.xls', './data/2012_06_Indicadores_Gestion.xls', './data/2012_03_Indicadores_Gestion.xls',
        ]

## Trabajando los Data Frames - Función cleanData()
* Creando una columna con fecha para cada Data Frame
* Creando columna con variable de Riesgos de cada dato 
* Reemplazando los datos incorrectos
* Eliminando columnas que no sirven
* Posibilidad de guardar en archivo csv

## Función completa

In [45]:
def enlace_df(enlaces):
    """
    What I need to work?
        - A list with the path's of the xls
        
    What I do? 
        - Turn the xls's in a list of data frames
    """
    l_dataframes = []
    
    for enlace in l_enlaces:
        df = pd.read_excel(enlace, sheet_name='1 Indicadores Patrimoniales', skiprows=3, header=1)
        l_dataframes.append(df)
    
    return l_dataframes


def nuevas_columnas(df, fecha):
    """
    What I need to work?
        - Data Frame
        - The number of date index to complete
        
    What I do? 
        - Fill the 'Fecha' colum with the correct date, so you can filter later in the análisis
    """
    
    fechas = ['06-2018', '03-2018',
        '12-2017', '09-2017', '06-2017', '03-2017',
        '12-2016', '09-2016', '06-2016', '03-2016',
        '12-2015', '09-2015', '06-2015', '03-2015',
        '12-2014', '09-2014', '06-2014', '03-2014',
        '12-2013', '09-2013', '06-2013', '03-2013',
        '12-2012', '09-2012', '06-2012', '03-2012',]
    
    df['Fecha'] = fechas[fecha]
    return df


def identificador_ramo(df):
    """
    What I need to work?
        - The Data Frame to fill
        
    What I do? 
        - Create the 'Categoría' column
        - Fill the column with the rigth category of insurens to future análisis
        - Restart de 'current_cat' variable so you can use this in a lot of Data Frames
    """  
    df['Categoría'] = None
    current_cat = 'Patrimoniales y Mixtas'
    
    for i in range(len(df)):
        if pd.isnull(df.iloc[i].NJ):
            current_cat = df.iloc[i]['Denominación']
        else:
            df.loc[i,'Categoría'] = current_cat
    
    current_cat = 'Patrimoniales y Mixtas'
    return df


def tirar_resetear(df):
    """
    What I need to work?
        - The Data Frame to clean
    What I do? 
        - Drop the rows you dont need
        - Reset the index
    """
    df.drop(df.index[[0,1,2,3,4,5]],inplace=True)
    df.reset_index(drop=True, inplace=True)
    
    return df

def a_categoria(df):
    """
    What I need to work?
        - The Data Frame
    What I do? 
        - Turn into Category the columns that you tell me
        - Drop the rows that you dont need
    """
    df['Categoría'] = df['Categoría'].astype('category', inplace=True)
    df['Denominación'] = df['Denominación'].astype('category', inplace=True)
    df = df[~df.Categoría.isnull()]
    df.reset_index(drop=True, inplace=True)
    
    return df

def no_nan(df):
    """
    What I need to work?
        - The Data Frame to clean
    What I do? 
        - Eliminate the null values, to avoid errors in the analisis 
    """
    df.replace('//',0,inplace=True)
    df.replace('///',0,inplace=True)
    return df

def filtro_ramo(df, ramo):
    """
    What I need to work?
        - The Data Frame
        - The insurance branch you want to filter
    What I do? 
        - 
    """
    ramos = {'PAT':'Patrimoniales y Mixtas', 
             'ART':'Riesgos del Trabajo', 
             'TRANSPORTE':'Transporte Público Pasajeros', 
             'VIDA':'Seguros de Vida', 
             'RETIRO':'Seguros de Retiro'}
    
    return df[ df['Categoría']== ramos[ramo]]
    
def cleanData(l_enlaces, ramo):
    """
    What I need to work?
        - A list with the xls files order in descending date
        - The insurance branch you want to filter
    What I do? 
        - Convert the xls in data frames
        - Clean the data frames
        - Separate by branch
        - Put it in a sigle csv that you can filter by branch or year
    """
    l_dataframes = enlace_df(l_enlaces)  # Lista de enlaces 
    largo = len(l_dataframes)            # Largo de la lista 
    
    df_Unido = pd.DataFrame()            # DF para guardar los df unidos
    

    for l, df in zip(range(largo),l_dataframes):
        
        tirar_resetear(df)               # Elimino las filas que no sirven y reseto el index

        nuevas_columnas(df,l)            # Agregando la columna "Fecha" y "Categoría"
        
        identificador_ramo(df)           # Coloco los ramos

        a_categoria(df)                  # Seteo como categorías las columnas y elimino las filas vacías

        no_nan(df)                       # Eliminamos los valores invalidos del Data Frame
        
        df_Unido = df_Unido.append(filtro_ramo(df, ramo), ignore_index=True)
    
    
    df_Unido.to_csv('./clean_data/{}2.csv'.format(ramo))
    return 


In [46]:
cleanData(l_enlaces,'PAT')

 **Posibles ramos a elegir en la función:**
* 'PAT' = Patrimoñales y Mixta
* 'ART' = Riesgos del Trabajo
* 'TRANSPORTE' = Transporte Público Pasajeros
* 'VIDA' = Seguros de Vida
* 'RETIRO' = Seguros de Retiro